<a href="https://colab.research.google.com/github/donghwi2022/ds-sa-cp2-phase2/blob/%EC%8B%A0%EB%8F%99%ED%9C%98/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 import 및 사용 데이터 load

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 모든 column을 확인하도록 설정
pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_parquet("/content/drive/MyDrive/cp2_phase2_data/fashion_logs.parquet")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201009 entries, 0 to 1254584
Data columns (total 23 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   created_at          1201009 non-null  category
 1   session_id          1201009 non-null  int32   
 2   payment_method      1201009 non-null  category
 3   promo_amount        1201009 non-null  int32   
 4   promo_code          1201009 non-null  category
 5   shipment_fee        1201009 non-null  int32   
 6   total_amount        1201009 non-null  int32   
 7   product_id          1201009 non-null  int32   
 8   productDisplayName  1201009 non-null  category
 9   quantity            1201009 non-null  int32   
 10  item_price          1201009 non-null  int32   
 11  product_gender      1201009 non-null  category
 12  baseColor           1201009 non-null  category
 13  season              1201009 non-null  category
 14  year                1201009 non-null  int32   
 15

In [6]:
data.head()

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,item_price,product_gender,baseColor,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
0,2018-07-29T15:22:01,194913,Debit Card,1415,WEEKENDSERU,10000,199832,54728,Vans Men Black Shoes,1,191247,Men,Black,Summer,2012,Casual,Footwear Shoes Casual Shoes,5868,F,2000-08-20,Android,Jakarta Raya,2018-07-28
1,2018-07-30T12:40:22,155874,Credit Card,0,No,10000,155526,16193,Puma Men Knitted Vest Green Sweater,1,145526,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
2,2018-09-15T11:51:17,489413,OVO,0,No,10000,550696,53686,Kiara Women Purple & Yellow Handbag,4,135174,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
3,2018-11-01T11:23:48,627839,Credit Card,0,No,0,271012,20228,Wrangler Women Cable Red Sweater,1,271012,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
4,2018-12-18T11:20:30,742846,Credit Card,0,No,0,198753,55220,Lakme Absolute Forever Silk Chestnut Lip Liner 03,1,198753,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28


# 2. 베이스라인 모델 구축
- 가장 인기있는 상위 10개 추천

## 1. 학습 데이터와 테스트 데이터 분리

In [40]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=2)

In [43]:
print("학습 데이터의 수 :", train_data.shape[0])
print("테스트 데이터의 수 :", test_data.shape[0])

학습 데이터의 수 : 960807
테스트 데이터의 수 : 240202


## 2. 빈도가 높은 상위 아이템 10개 추출

In [44]:
# 빈도가 높은 상위 10개 아이템 추출
popular_item = train_data['product_id'].value_counts()
top_10 = popular_item.head(10)
top_10

54932    45
15637    43
57252    42
40615    41
21753    40
34930    40
28084    40
1846     40
4899     39
7548     39
Name: product_id, dtype: int64

In [51]:
# 데이터 프레임으로 변경
recomm_data = pd.DataFrame({'product_id' : top_10.index ,
              'product_amount' : top_10.values})
recomm_data

,product_id,product_amount
0,54932,45
1,15637,43
2,57252,42
3,40615,41
4,21753,40
5,34930,40
6,28084,40
7,1846,40
8,4899,39
9,7548,39


# 3. 성능 평가

In [47]:
# precision@k 계산 함수
def precision_k(test_data, recommend_data) : 
    recommend_list = recommend_data['product_id'].to_list()
    common_product = test_data[test_data['product_id'].isin(recommend_list)]

    return len(common_product) / len(recommend_list)

# recall@k 계산 함수 
def recall_k(test_data, recommend_data) : 
    recommend_list = recommend_data['product_id'].to_list()
    common_product = test_data[test_data['product_id'].isin(recommend_list)]

    return len(common_product) / len(test_data)

In [58]:
# 고객 id가 29496인 샘플 데이터로 테스트
sample_data = test_data[test_data['customer_id'] == 29496]

print(precision_k(sample_data, recomm_data))
print(recall_k(sample_data, recomm_data))

0.1
0.006993006993006993


In [62]:
# 모든 고객 id에 대해서 테스트
customer_list = test_data['customer_id'].unique()
precision_total = 0
recall_total = 0

for customer_id in customer_list : 
    sample_data = test_data[test_data['customer_id'] == customer_id]
    precision_total += precision_k(sample_data, recomm_data)
    recall_total += recall_k(sample_data, recomm_data)

print("precision :", precision_total/len(customer_list))
print("reall :", recall_total/len(customer_list))

precision : 0.00013508949679162443
reall : 0.00014297477655136664
